<a href="https://colab.research.google.com/github/mdapoy/Machine-Learning-week-8-16/blob/main/Ch13_Loading_and_Preprocessing_Data_with_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laporan Analisis Bab 13: Loading and Preprocessing Data with TensorFlow

## Pendahuluan
Bab 13 berfokus pada penanganan dataset besar yang tidak muat di memori (RAM) dalam konteks Deep Learning. Ini adalah tantangan umum karena model Deep Learning modern seringkali dilatih dengan volume data yang sangat besar. Bab ini memperkenalkan TensorFlow Data API sebagai solusi efisien untuk memuat dan memproses data, serta membahas format TFRecord untuk penyimpanan data yang optimal. Selain itu, bab ini mengeksplorasi teknik-teknik pra-pemrosesan fitur, termasuk penanganan fitur kategorikal dan teks, serta memperkenalkan lapisan pra-pemrosesan Keras dan alat TF Transform.

## Ringkasan Isi Bab

Bab 13 menguraikan beberapa konsep kunci terkait Pemuatan dan Pra-pemrosesan Data dengan TensorFlow:

1.  **The Data API (API Data)**:
    * **Konsep Dasar**: Seluruh Data API berpusat pada konsep `dataset`, yang merepresentasikan urutan item data. `tf.data.Dataset.from_tensor_slices()` dapat membuat dataset dari *tensor* di RAM.
    * **Chaining Transformations (Menggabungkan Transformasi)**: Setelah membuat dataset, berbagai transformasi dapat diterapkan secara berantai menggunakan metode dataset seperti `repeat()`, `batch()`, `map()`, `apply()`, dan `filter()`. Setiap metode mengembalikan dataset baru, memungkinkan alur pemrosesan yang fleksibel. Fungsi yang diberikan ke `map()` harus dapat dikonversi menjadi TF Function.
    * **Shuffling the Data (Mengocok Data)**: `shuffle()` digunakan untuk mengocok instance, yang penting agar instance pelatihan independen dan terdistribusi secara identik (IID). Perlu ditentukan ukuran *buffer* yang cukup besar. Untuk dataset yang sangat besar, dapat mengocok data sumber atau menginterleave baris dari banyak file acak.
    * **Interleaving lines from multiple files (Menginterleave baris dari banyak file)**: `tf.data.Dataset.list_files()` membuat dataset dari *filepath*. Metode `interleave()` dapat membaca dari beberapa file secara bersamaan dan menggabungkan baris-barisnya, seringkali dengan `tf.data.TextLineDataset` dan `skip()` untuk melewati *header*. Ini mendukung paralelisasi dengan `num_parallel_calls`.
    * **Preprocessing the Data (Pra-pemrosesan Data)**: Dapat dilakukan dengan metode `map()` menggunakan fungsi pra-pemrosesan kustom. Fungsi ini harus menggunakan operasi TensorFlow dan dapat menskalakan fitur.
    * **Putting Everything Together (Menggabungkan Semuanya)**: Contoh fungsi `csv_reader_dataset` menunjukkan bagaimana menggabungkan `list_files()`, `interleave()`, `map()`, `shuffle()`, `repeat()`, `batch()`, dan `prefetch()` untuk membuat *input pipeline* yang efisien.
    * **Prefetching (Pra-pengambilan)**: `prefetch(1)` di akhir *pipeline* membuat dataset selalu satu *batch* di depan. Ini memungkinkan CPU dan GPU bekerja secara paralel, secara dramatis meningkatkan kinerja pelatihan. Untuk dataset yang muat di memori, `cache()` dapat mempercepat pelatihan dengan menyimpan konten dataset di RAM setelah pra-pemrosesan.
    * **Using the Dataset with tf.keras (Menggunakan Dataset dengan tf.keras)**: Dataset yang dibuat dengan Data API dapat langsung diumpankan ke metode `fit()`, `evaluate()`, dan `predict()` dari model Keras. Ini juga memungkinkan iterasi langsung pada dataset dalam *custom training loops*.

2.  **The TFRecord Format (Format TFRecord)**:
    * **Definisi**: Format biner pilihan TensorFlow untuk menyimpan data dalam jumlah besar dan membacanya secara efisien. Berisi urutan catatan biner dengan ukuran bervariasi.
    * **Membuat File TFRecord**: Menggunakan `tf.io.TFRecordWriter`.
    * **Membaca File TFRecord**: Menggunakan `tf.data.TFRecordDataset`. Dapat membaca banyak file secara paralel dengan `num_parallel_reads`.
    * **Compressed TFRecord Files (File TFRecord Terkompresi)**: Mendukung kompresi (misalnya, GZIP) untuk mengurangi ukuran file, berguna saat memuat melalui jaringan.
    * **Introduction to Protocol Buffers (Pengantar Protocol Buffers - Protobufs)**:
        * Format biner portabel, dapat diperluas, dan efisien yang dikembangkan oleh Google. Didefinisikan menggunakan bahasa sederhana (`.proto`).
        * Data dalam file TFRecord umumnya berisi *serialized protocol buffers*, terutama *Example* protobuf.
        * Contoh dasar penggunaan kelas akses protobuf Python (misalnya, `Person` protobuf, `SerializeToString()`, `ParseFromString()`).
    * **TensorFlow Protobufs**:
        * `Example` protobuf: Merepresentasikan satu instance dalam dataset, berisi daftar fitur bernama (byte strings, float, atau integer). Fitur-fitur ini disimpan dalam objek `Features`.
        * `tf.train.Example`: Kelas untuk membuat *Example* protobuf.
        * **Loading and Parsing Examples (Memuat dan Mem-parsing Contoh)**: Menggunakan `tf.io.parse_single_example()` (untuk satu catatan) atau `tf.io.parse_example()` (untuk *batch*) dengan deskripsi fitur (`tf.io.FixedLenFeature` atau `tf.io.VarLenFeature`).
        * **Menyimpan Data Kompleks**: `BytesList` dapat menyimpan data biner apa pun (misalnya, gambar JPEG atau *tensor* yang diserialisasi).
        * `SequenceExample` protobuf: Dirancang untuk kasus penggunaan daftar-daftar (misalnya, dokumen teks dengan kalimat dan kata-kata). Diparse dengan `tf.io.parse_single_sequence_example()` atau `tf.io.parse_sequence_example()`.

3.  **Preprocessing the Input Features (Pra-pemrosesan Fitur Input)**:
    * **Tujuan**: Mengubah semua fitur menjadi numerik, menormalisasikannya, dan mengodekan fitur kategorikal atau teks.
    * **Implementasi Pra-pemrosesan**: Dapat dilakukan dengan metode `map()` dataset atau langsung dalam model sebagai lapisan pra-pemrosesan.
    * **Standardization Layer (Lapisan Standardisasi)**: Implementasi kustom dapat dibuat sebagai subclass `keras.layers.Layer` dengan metode `adapt()` (untuk menghitung rata-rata/std dari sampel data) dan `call()` (untuk menerapkan transformasi). Keras kemungkinan akan menyediakan `keras.layers.Normalization`.
    * **Encoding Categorical Features Using One-Hot Vectors (Mengodekan Fitur Kategorikal Menggunakan One-Hot Vectors)**:
        * Menggunakan tabel pencarian (`tf.lookup.StaticVocabularyTable`) untuk memetakan kategori ke ID integer, termasuk *out-of-vocabulary* (OOV) *buckets* untuk kategori yang tidak dikenal.
        * Kemudian `tf.one_hot()` mengodekan ID menjadi *one-hot vector*.
        * Cocok untuk kategori < 10.
    * **Encoding Categorical Features Using Embeddings (Mengodekan Fitur Kategorikal Menggunakan Embeddings)**:
        * *Embedding* adalah *dense vector* yang dapat dilatih dan merepresentasikan kategori.
        * `keras.layers.Embedding` layer menangani matriks *embedding* (dapat dilatih).
        * *Embeddings* akan membaik selama pelatihan, dan kategori serupa akan memiliki *embedding* yang lebih dekat (*representation learning*).
        * *Word Embeddings*: Contoh paling umum, seringkali dapat digunakan kembali dari model *pretrained* (misalnya, dari TensorFlow Hub).
        * Cocok untuk kategori > 50.
    * **Keras Preprocessing Layers (Lapisan Pra-pemrosesan Keras)**:
        * Tim TensorFlow sedang mengembangkan serangkaian lapisan pra-pemrosesan standar Keras (misalnya, `keras.layers.Normalization`, `keras.layers.TextVectorization`, `keras.layers.Discretization`).
        * Metode `adapt()` akan mengekstrak kosakata/statistik dari sampel data.
        * `PreprocessingStage` dapat menggabungkan beberapa lapisan pra-pemrosesan.
        * `TextVectorization` dapat menghasilkan *word-count vectors* (*bag of words*) atau TF-IDF.
        * Lapisan pra-pemrosesan ini harus digunakan di awal model dan akan dibekukan selama pelatihan.

4.  **TF Transform (tf.Transform)**:
    * Bagian dari TensorFlow Extended (TFX), platform *end-to-end* untuk produksi model TensorFlow.
    * **Tujuan**: Menulis fungsi pra-pemrosesan hanya sekali (di Python menggunakan fungsi TF Transform). Fungsi ini dapat dijalankan dalam mode *batch* pada seluruh dataset pelatihan sebelum pelatihan (untuk kecepatan).
    * **Deployment**: TF Transform akan menghasilkan TF Function yang setara yang dapat dimasukkan ke dalam model yang sudah dilatih dan diterapkan di produksi, sehingga menghindari *training/serving skew* (perbedaan pra-pemrosesan antara pelatihan dan produksi). Ini mengurangi pemeliharaan kode pra-pemrosesan di berbagai platform.

5.  **The TensorFlow Datasets (TFDS) Project (Proyek TFDS)**:
    * **Tujuan**: Memudahkan pengunduhan dan pemuatan dataset umum (dari kecil seperti MNIST hingga besar seperti ImageNet).
    * **Penggunaan**: `tfds.load(name="mnist", as_supervised=True)` akan mengunduh data dan mengembalikannya sebagai `tf.data.Dataset` dalam format terawasi (fitur, label).
    * Dataset TFDS dapat langsung digunakan dengan model `tf.keras` setelah transformasi yang diperlukan (misalnya, `shuffle()`, `batch()`, `prefetch()`).

## Analisis dan Relevansi untuk Mahasiswa

Bab 13 adalah fondasi kritis bagi mahasiswa yang akan berhadapan dengan dataset besar dan kompleks dalam proyek Deep Learning mereka.

* **Pentingnya *Input Pipeline***: Mahasiswa akan memahami bahwa *input pipeline* yang efisien sama pentingnya dengan arsitektur model dalam mencapai kinerja pelatihan yang optimal, terutama untuk dataset yang tidak muat di memori.
* **Penguasaan Data API**: Detail tentang `tf.data.Dataset` dan berbagai transformasinya adalah pengetahuan inti. Ini membekali mahasiswa dengan kemampuan untuk membangun alur data yang fleksibel, paralel, dan berkinerja tinggi.
* **Penyimpanan Data Optimal**: Pengenalan TFRecord dan Protocol Buffers mengajarkan mahasiswa format penyimpanan data yang direkomendasikan TensorFlow untuk efisiensi skala besar, penting untuk proyek yang melampaui CSV sederhana.
* **Pra-pemrosesan yang Robust**: Bab ini menyediakan berbagai strategi untuk menangani fitur numerik, kategorikal, dan teks. Mahasiswa akan belajar cara mengodekan fitur, menskalakan data, dan memilih antara pra-pemrosesan *offline* atau *on-the-fly*.
* **Mencegah *Training/Serving Skew***: Konsep *training/serving skew* dan solusi yang ditawarkan oleh lapisan pra-pemrosesan Keras dan TF Transform adalah wawasan tingkat lanjut yang krusial untuk penerapan model di dunia nyata. Ini mendorong praktik rekayasa ML yang baik.
* **Sumber Dataset yang Mudah**: TFDS menyederhanakan proses mendapatkan dan menggunakan dataset publik, memungkinkan mahasiswa untuk lebih fokus pada eksperimen model daripada akuisisi data.

## Kesimpulan

Bab 13 memberikan panduan komprehensif untuk mengelola, memuat, dan mempra-memproses data secara efisien di TensorFlow. Mahasiswa yang menguasai bab ini akan sangat siap untuk menghadapi tantangan data skala besar dan membangun *input pipeline* yang solid untuk model Deep Learning mereka. Kemampuan ini merupakan keterampilan praktis yang tidak terpisahkan dari pengembangan sistem Machine Learning yang efektif dan dapat diterapkan di produksi.

In [10]:
# Import library yang diperlukan
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os

# Mengatur seed untuk reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# --- Persiapan Data California Housing (untuk contoh utama) ---
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

print("--- Mempersiapkan Data California Housing ---")
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

# Skala data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

# Membuat direktori 'datasets/housing'
housing_dir = os.path.join("datasets", "housing")
os.makedirs(housing_dir, exist_ok=True)

def save_to_csv(filepath, X, y, header=None, mode="w"):
    with open(filepath, mode) as f:
        if header is not None:
            f.write(header + "\n")
        for row, target in zip(X, y):
            f.write(",".join([str(x) for x in row]) + "," + str(target) + "\n")

# Simpan data MENTAH ke CSV, penskalaan akan dilakukan di fungsi preprocess
header_cols = housing.feature_names + ["MedianHouseValue"]
header = ",".join(header_cols)
train_data_path = os.path.join(housing_dir, "my_train.csv")
valid_data_path = os.path.join(housing_dir, "my_valid.csv")
test_data_path = os.path.join(housing_dir, "my_test.csv")
save_to_csv(train_data_path, X_train_full, y_train_full, header=header)
save_to_csv(valid_data_path, X_valid, y_valid, header=header)
save_to_csv(test_data_path, X_test, y_test, header=header)

# Memecah file training
train_filepaths = []
for i in range(10):
    filepath = os.path.join(housing_dir, f"my_train_{i:02d}.csv")
    save_to_csv(filepath, X_train_full[i::10], y_train_full[i::10], header=header)
    train_filepaths.append(filepath)

valid_filepaths = [valid_data_path]
test_filepaths = [test_data_path]
print("Data California Housing telah disimpan ke file CSV.")


# --- BAGIAN 1: The Data API ---
print("\n--- Bagian 1: The Data API ---")
X_mean = X_train_full.mean(axis=0, keepdims=True)
X_std = X_train_full.std(axis=0, keepdims=True)
n_inputs = X_train_full.shape[1]

def preprocess(line):
    defs = [0.0] * n_inputs + [tf.constant(0.0, dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(fields[:-1])
    y = fields[-1]
    scaled_x = (x - X_mean) / X_std
    return tf.squeeze(scaled_x), y

def csv_reader_dataset(filepaths, repeat=1, shuffle_buffer_size=10000, batch_size=32):
    dataset = tf.data.Dataset.list_files(filepaths, seed=42)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=5, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle_buffer_size > 0:
        dataset = dataset.shuffle(shuffle_buffer_size, seed=42)
    dataset = dataset.repeat(repeat)
    return dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

batch_size = 32
train_set = csv_reader_dataset(train_filepaths, batch_size=batch_size, repeat=None)
valid_set = csv_reader_dataset(valid_filepaths, batch_size=batch_size)
test_set = csv_reader_dataset(test_filepaths, batch_size=batch_size)

model_keras_dataset = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[n_inputs]),
    keras.layers.Dense(1)
])
model_keras_dataset.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

steps_per_epoch = len(X_train_full) // batch_size
history_dataset = model_keras_dataset.fit(train_set, epochs=2,
                                          steps_per_epoch=steps_per_epoch,
                                          validation_data=valid_set)


# --- BAGIAN 2: The TFRecord Format ---
print("\n--- Bagian 2: The TFRecord Format ---")
# (Bagian ini tidak diubah dan diasumsikan sudah benar)
from tensorflow.train import BytesList, FloatList, Int64List, Feature, Features, Example

# ... (Kode TFRecord dari sebelumnya) ...


# --- BAGIAN 3: Preprocessing the Input Features ---
print("\n--- Bagian 3: Preprocessing the Input Features ---")

# 3.1 Custom Standardization Layer
class Standardization(keras.layers.Layer):
    def adapt(self, data_sample):
        self.means_ = np.mean(data_sample, axis=0, keepdims=True)
        self.stds_ = np.std(data_sample, axis=0, keepdims=True)
    def call(self, inputs):
        return (inputs - self.means_) / (self.stds_ + keras.backend.epsilon())

# 3.2 Encoding Categorical Features Using One-Hot Vectors
vocab = ["<1H OCEAN", "INLAND", "NEAR OCEAN", "NEAR BAY", "ISLAND"]
indices = tf.range(len(vocab), dtype=tf.int64)
table_init = tf.lookup.KeyValueTensorInitializer(keys=vocab, values=indices)
num_oov_buckets = 2
table = tf.lookup.StaticVocabularyTable(table_init, num_oov_buckets)

# ##################################################################
# #############   PERBAIKAN UTAMA ADA DI BAGIAN INI   ##############
# ##################################################################
# 3.3 Encoding Categorical Features Using Embeddings
print("\n3.3 Encoding Categorical Features Using Embeddings:")
embedding_dim = 2

# Definisikan input untuk model fungsional
regular_inputs = keras.layers.Input(shape=[8])
categories_input = keras.layers.Input(shape=[], dtype=tf.string)

# PERBAIKAN: Bungkus operasi TensorFlow 'table.lookup' di dalam Lambda layer
# agar dapat bekerja dengan Keras Input layer (symbolic tensor).
cat_indices_model = keras.layers.Lambda(lambda cats: table.lookup(cats))(categories_input)

cat_embed = keras.layers.Embedding(
    input_dim=len(vocab) + num_oov_buckets,
    output_dim=embedding_dim
)(cat_indices_model)

# Gabungkan kembali input numerik dan embedding
encoded_inputs = keras.layers.concatenate([regular_inputs, cat_embed])

outputs_model_embedding = keras.layers.Dense(1)(encoded_inputs)
model_with_embedding = keras.models.Model(inputs=[regular_inputs, categories_input],
                                          outputs=[outputs_model_embedding])

print("Model dengan Embedding Layer untuk fitur kategorikal berhasil dibuat.")
model_with_embedding.summary()
# ##################################################################
# ##################################################################


# --- BAGIAN 4: TF Transform (Konseptual) ---
print("\n--- Bagian 4: TF Transform (Konseptual) ---")
try:
    import tensorflow_transform as tft
    print("Paket tensorflow-transform ditemukan.")
except ImportError:
    print("Paket tensorflow-transform tidak ditemukan. Melewati bagian 4.")


# --- BAGIAN 5: The TensorFlow Datasets (TFDS) Project ---
print("\n--- Bagian 5: The TensorFlow Datasets (TFDS) Project ---")
try:
    import tensorflow_datasets as tfds

    print("\nMemuat dataset MNIST dari TFDS...")
    dataset_mnist_tfds, info_mnist_tfds = tfds.load(name="mnist", as_supervised=True, with_info=True)
    mnist_train_tfds, mnist_test_tfds = dataset_mnist_tfds["train"], dataset_mnist_tfds["test"]
    print("Dataset MNIST dari TFDS dimuat.")

    model_tfds = keras.models.Sequential([
        keras.layers.Flatten(input_shape=[28, 28, 1]),
        keras.layers.Lambda(lambda x: tf.cast(x, tf.float32) / 255.0),
        keras.layers.Dense(100, activation="relu"),
        keras.layers.Dense(10, activation="softmax")
    ])
    model_tfds.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

    print("\nMelatih model Keras dengan TFDS dataset (singkat):")
    model_tfds.fit(mnist_train_tfds.shuffle(10000, seed=42).batch(32).prefetch(tf.data.AUTOTUNE),
                   epochs=2,
                   validation_data=mnist_test_tfds.batch(32).prefetch(tf.data.AUTOTUNE))
    print("Pelatihan dengan TFDS dataset selesai.")

except ImportError:
    print("\nPaket tensorflow-datasets tidak ditemukan. Melewati bagian 5.")

print("\n--- Skrip selesai dieksekusi dengan perbaikan. ---")

--- Mempersiapkan Data California Housing ---
Data California Housing telah disimpan ke file CSV.

--- Bagian 1: The Data API ---


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/2
483/483 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 2.6802 - val_loss: 0.8408
Epoch 2/2
 48/483 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.7770

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


483/483 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.7877 - val_loss: 0.6900


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



--- Bagian 2: The TFRecord Format ---

--- Bagian 3: Preprocessing the Input Features ---

3.3 Encoding Categorical Features Using Embeddings:


NotImplementedError: Exception encountered when calling Lambda.call().

[1mWe could not automatically infer the shape of the Lambda's output. Please specify the `output_shape` argument for this Lambda layer.[0m

Arguments received by Lambda.call():
  • args=('<KerasTensor shape=(None,), dtype=string, sparse=False, name=keras_tensor_36>',)
  • kwargs={'mask': 'None'}